In [1]:
data_var = '2023-12-12'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7689,2023-12-12,Brasil Nbb,20:00,São José,Mogi,1.17,4.56,156.5,1.87,1.79,-11.5,1.98,1.74,lnxkX89T,0.854701,0.219298,0.534759,0.558659,0.073999,0.6,0.4,NaN,NaN,123.068,14.351981,0.116618,254.626,139.198616,0.546679,195.062,202.884,145.25,254.61,0.0,0.0,0.0,0.0,0.836681,0.030912,0.091240,3.15,0.630,0.269841,0.000000,0.0,0.000000,-1.56,-0.312,-11.410256,0.000000,0.0,0.000000
7690,2023-12-12,Espanha Acb,16:30,Baskonia,Palencia,1.06,9.40,165.5,1.83,1.93,-16.5,2.07,1.70,6mKN8vEl,0.943396,0.106383,0.546448,0.518135,0.049779,0.6,0.0,NaN,NaN,131.344,15.922031,0.121224,296.086,108.375445,0.366027,126.314,0.000,138.32,265.20,0.0,0.0,0.0,0.0,1.127585,0.037612,0.138795,-0.81,-0.162,-0.370370,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7691,2023-12-12,Eua Nba,21:30,Boston Celtics,Cleveland Cavaliers,1.21,5.31,222.5,1.80,2.08,-10.5,2.01,1.93,GjB4xXA8,0.826446,0.188324,0.555556,0.480769,0.014770,0.8,0.2,NaN,NaN,156.222,20.495800,0.131197,174.686,43.920933,0.251428,162.262,223.638,186.20,206.80,0.0,0.0,0.0,0.0,0.889306,0.102057,0.028715,1.60,0.320,0.656250,0.781683,0.9,0.118317,2.91,0.582,7.405498,0.526618,0.6,0.073382
7692,2023-12-12,Eua Nba,21:30,Dallas Mavericks,Los Angeles Lakers,2.11,1.85,232.5,1.80,2.08,1.5,2.01,1.93,fsA8yiQE,0.473934,0.540541,0.555556,0.480769,0.014474,0.6,0.8,NaN,NaN,192.350,61.790664,0.321241,234.614,69.083917,0.294458,195.960,237.066,192.57,333.30,0.0,0.0,0.0,0.0,0.092852,0.102057,0.028715,-2.19,-0.438,-2.534247,0.644552,0.5,-0.144552,-0.19,-0.038,-22.368421,0.505572,0.3,-0.205572
7693,2023-12-12,Eua Nba,22:00,Chicago Bulls,Denver Nuggets,3.38,1.39,218.5,1.80,2.08,6.5,2.09,1.85,x6aDzBuL,0.295858,0.719424,0.555556,0.480769,0.015282,0.6,0.6,NaN,NaN,376.600,234.171584,0.621805,268.878,153.296012,0.570132,272.424,268.760,173.16,225.75,0.0,0.0,0.0,0.0,0.589997,0.102057,0.086145,4.97,0.994,2.394366,0.521151,0.5,-0.021151,1.45,0.290,1.344828,0.553428,0.4,-0.153428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7752,2023-12-12,Suécia Basketligan,15:04,Umeå IK,Koping Stars,2.05,1.70,167.5,1.80,1.86,1.5,1.91,1.80,65NjZbOi,0.487805,0.588235,0.555556,0.537634,0.076040,0.0,0.0,NaN,NaN,753.764,294.676481,0.390940,287.292,107.510591,0.374221,0.000,0.000,268.32,439.24,0.0,0.0,0.0,0.0,0.131993,0.023184,0.041931,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7753,2023-12-12,Suécia Basketligan,15:04,Uppsala,BC Lulea,2.31,1.57,172.5,1.85,1.85,2.5,1.99,1.73,SEOnzu9o,0.432900,0.636943,0.540541,0.540541,0.069843,0.0,0.0,NaN,NaN,253.438,172.018400,0.678740,184.724,47.976766,0.259721,0.000,0.000,114.48,220.00,0.0,0.0,0.0,0.0,0.269721,0.000000,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7754,2023-12-12,Suécia Superettan,15:00,Norrort,AIK Basket,2.93,1.37,158.5,1.87,1.79,5.5,1.97,1.74,0pQnM7DF,0.341297,0.729927,0.534759,0.558659,0.071224,0.0,0.0,NaN,NaN,NaN,NaN,NaN,160.344,72.044421,0.449312,0.000,0.000,141.95,80.52,0.0,0.0,0.0,1.0,0.513064,0.030912,0.087674,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7755,2023-12-12,Uruguai Liga Uruguaia,21:00,Malvin,Larre Borges,1.14,5.19,165.5,1.86,1.84,-12.5,1.98,1.74,4bUCxSYq,0.877193,0.192678,0.537634,0.543478,0.069871,0.0,0.0,NaN,NaN,144.964,60.826836,0.419600,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,21:30,Eua Nba,Boston Celtics,Cleveland Cavaliers,222.5,1.80,1.0000,Over
3,21:30,Eua Nba,Dallas Mavericks,Los Angeles Lakers,232.5,1.80,1.0000,Over
4,22:00,Eua Nba,Chicago Bulls,Denver Nuggets,218.5,1.80,0.9999,Over
13,14:00,Croácia Copa Da Croácia,Cibona,Skrljevo,157.5,1.87,1.0000,Over
